In [1]:
from __future__ import annotations
import sqlparse

In [33]:
raw =  """SELECT *, clip(image_column, 'a dog walked across the street') as scores FROM df
WHERE a = 1 AND b = 2 AND c = 3"""

In [34]:
out = sqlparse.parse(raw)[0]
out.tokens

[<DML 'SELECT' at 0x109D4A8E0>,
 <Whitespace ' ' at 0x109D4A820>,
 <IdentifierList '*, cli...' at 0x109D53660>,
 <Whitespace ' ' at 0x109C7F460>,
 <Keyword 'FROM' at 0x109D58D60>,
 <Whitespace ' ' at 0x109D58E20>,
 <Identifier 'df' at 0x109D53CF0>,
 <Newline ' ' at 0x109D587C0>,
 <Where 'WHERE ...' at 0x109D53A50>]

In [35]:
out.tokens[2].tokens[-1]

<Identifier 'clip(i...' at 0x109D537B0>

In [53]:
def search_alias(parsed: sqlparse.sql.Statement):
    for token in parsed.tokens:
        if isinstance(token, sqlparse.sql.Identifier):
            return token
    
def find_all_clips(parsed: sqlparse.sql.Statement):
    clip_tokens = []
    def _find_all_clips(parsed: sqlparse.sql.Statement):
        try:
            alias = search_alias(parsed)
            tokens = parsed.tokens
            for token in tokens:
                if isinstance(token, sqlparse.sql.Function) and token.get_name().lower() == 'clip':
                    # replace the actual clip with just the alias now (since I compute and store it here in a different call)
                    parsed.tokens = [alias]
                    clip_tokens.append((token, alias.value))
                else:
                    _find_all_clips(token)
        except:
            pass
        
    _find_all_clips(parsed)

    return clip_tokens

find_all_clips(out)

[]

In [61]:

def parse_clip_fn(parsed) -> tuple[str, str]:
    # this shit is broke, use tokens instead to do this more accurately
    clip_fn, alias = parsed
    raw = clip_fn.tokens[1].value
    image_column = raw.strip("(").split(',')[0].strip()
    text = raw.strip(")").split(',')[1].strip().strip("'")
    return (image_column, text, alias )


def parse_clip_query(sql_raw: str):
    # just one statement for now
    parsed = sqlparse.parse(sql_raw)[0]

    # create a list of ClipQueryExecutor objects parsed from
    # clip(column_name, 'text') as scores clauses
    clip_fns = find_all_clips(parsed)
    clip_executors = [parse_clip_fn(clip_fn) for clip_fn in clip_fns]
    return clip_executors, str(parsed)

parsed, new_sql = parse_clip_query(raw)

In [62]:
parsed

[('image_column', 'a dog walked across the street', 'scores')]

In [63]:
new_sql

'SELECT *, scores FROM df\nWHERE a = 1 AND b = 2 AND c = 3'